In [1]:
import arcpy
import os

In [6]:
import os
import arcpy

# -----------------------------
# Paths
# -----------------------------
path_16    = r"D:\Users\abolmaal\data\coastalwetlands\UTM16_new"
path_17    = r"D:\Users\abolmaal\data\coastalwetlands\UTM17_new\UTM17_new\MI_Huron_utm17N"
final_path = r"D:\Users\abolmaal\data\coastalwetlands\finalwetland"

os.makedirs(final_path, exist_ok=True)
arcpy.env.overwriteOutput = True

# Categories
categories = ["avg", "high", "low", "surge"]

# -----------------------------
# Spatial references
# -----------------------------
# Pick the right UTM CRS for your data:
# If your UTM files are NAD83 UTM:
sr_utm16 = arcpy.SpatialReference(26916)  # NAD83 / UTM zone 16N
sr_utm17 = arcpy.SpatialReference(26917)  # NAD83 / UTM zone 17N

# (If they are WGS84 UTM instead, use 32616 / 32617.)

# Final projection: Great Lakes Albers
sr_gl_albers = arcpy.SpatialReference(3174)  # NAD83 / Great Lakes Albers

# -----------------------------
# Helper: define projection if missing
# -----------------------------
def ensure_projection(fc_path, sr_target):
    desc = arcpy.Describe(fc_path)
    if (desc.spatialReference is None or
        desc.spatialReference.name in ("Unknown", "", None)):
        print(f"Defining projection for {fc_path} -> {sr_target.name}")
        arcpy.DefineProjection_management(fc_path, sr_target)
    else:
        print(f"Existing spatial reference for {fc_path}: {desc.spatialReference.name}")

# -----------------------------
# Main loop
# -----------------------------
for cat in categories:
    in_utm16 = os.path.join(path_16, f"wetlands_connected_{cat}_inundation_utm16.shp")
    in_utm17 = os.path.join(path_17, f"wetlands_connected_{cat}_inundation_utm17.shp")

    if not arcpy.Exists(in_utm16):
        print(f"⚠ UTM16 file missing for {cat}: {in_utm16}")
        continue
    if not arcpy.Exists(in_utm17):
        print(f"⚠ UTM17 file missing for {cat}: {in_utm17}")
        continue

    # 1) Ensure each input has a defined UTM CRS
    ensure_projection(in_utm16, sr_utm16)
    ensure_projection(in_utm17, sr_utm17)

    # 2) Project each to Great Lakes Albers (3174) into in_memory
    proj_16 = os.path.join("in_memory", f"wetlands_{cat}_utm16_glalbers")
    proj_17 = os.path.join("in_memory", f"wetlands_{cat}_utm17_glalbers")

    arcpy.Project_management(in_utm16, proj_16, sr_gl_albers)
    arcpy.Project_management(in_utm17, proj_17, sr_gl_albers)

    # 3) Merge projected feature classes
    out_fc = os.path.join(final_path, f"wetlands_connected_{cat}_inundation_GLAlbers.shp")
    arcpy.Merge_management([proj_16, proj_17], out_fc)

    # Clean up in_memory
    arcpy.Delete_management(proj_16)
    arcpy.Delete_management(proj_17)

    print(f"✅ Merged {cat} to Great Lakes Albers: {out_fc}")


Existing spatial reference for D:\Users\abolmaal\data\coastalwetlands\UTM16_new\wetlands_connected_avg_inundation_utm16.shp: WGS_1984_UTM_Zone_16N
Existing spatial reference for D:\Users\abolmaal\data\coastalwetlands\UTM17_new\UTM17_new\MI_Huron_utm17N\wetlands_connected_avg_inundation_utm17.shp: NAD_1983_UTM_Zone_17N
✅ Merged avg to Great Lakes Albers: D:\Users\abolmaal\data\coastalwetlands\finalwetland\wetlands_connected_avg_inundation_GLAlbers.shp
Existing spatial reference for D:\Users\abolmaal\data\coastalwetlands\UTM16_new\wetlands_connected_high_inundation_utm16.shp: WGS_1984_UTM_Zone_16N
Existing spatial reference for D:\Users\abolmaal\data\coastalwetlands\UTM17_new\UTM17_new\MI_Huron_utm17N\wetlands_connected_high_inundation_utm17.shp: NAD_1983_UTM_Zone_17N
✅ Merged high to Great Lakes Albers: D:\Users\abolmaal\data\coastalwetlands\finalwetland\wetlands_connected_high_inundation_GLAlbers.shp
Existing spatial reference for D:\Users\abolmaal\data\coastalwetlands\UTM16_new\wetlan